In [2]:
!pip install tqdm --user

In [1]:
from tqdm import tqdm
import argparse
import math
import os
import json
import pickle
from scipy import sparse
from scipy import constants as const
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from random import *
import numpy as np
import time
from ROOT import TFile
from datetime import datetime

Welcome to JupyROOT 6.18/00


In [2]:
signal=False
filename = ""
if signal:
    FILEPATH = "/global/cfs/cdirs/majorana/data/mjd/surfmjd/analysis/skim/DS6/GAT-v02-03-54-g4ecb54c"
    label = 1
    filename="DS6_6.csv"
else:
    FILEPATH = "/global/cfs/cdirs/majorana/data/mjd/surfmjd/analysis/skim/DS6cal/GAT-v02-03-54-g4ecb54c"
    label = 0
    filename="DS6cal_6.csv"

In [ ]:
#Only using high gain channel
count=0
with open(filename, "w") as f:
    for file in tqdm(os.listdir(FILEPATH)):
        f1 = TFile(os.path.join(FILEPATH, file))
        tree = f1.Get("skimTree")
        filestring = ""
        for evt_index in range(tree.GetEntries()):
            tree.GetEntry(evt_index)
            if (tree.Final_Energy[0] < 1500.0) and (tree.Final_Energy[0] != 0.0):
                continue
            if signal:
                passflag = np.all(np.array(tree.isGood)) and tree.isLNFill1==0 and tree.isLNFill2==0 and np.array(tree.wfDCBits).all()==0 and tree.mHL==1 and tree.muVeto==0
            else:
                passflag = np.all(np.array(tree.isGood)) and tree.isLNFill1==0 and tree.isLNFill2==0 and np.array(tree.wfDCBits).all()==0  and tree.muVeto==0  and (tree.mH==1 and tree.mL==1)
            if passflag:
                chindex = 0
                if not signal:
#                     if np.random.rand()>0.1:
#                         continue
                    if tree.sumEHClean == 0.0:
                        chindex = 1
                count+=1
                f.writelines("%d,%d,%f,%f,%f,%f,%f,%d,%f\n"%(int(np.array(tree.isEnr,dtype=np.bool)[chindex]),\
                                                                tree.channel[chindex],\
                                                                tree.tDrift[chindex],\
                                                                tree.Final_Energy[chindex],\
                                                                tree.avse[chindex],\
                                                                tree.dcr[chindex],\
                                                                tree.d2wfnoiseTagNorm[chindex],\
                                                                tree.mAct_g[chindex],\
                                                                tree.avse_corr[chindex]
                                                                ))
print(count)

 11%|█         | 493/4414 [1:16:32<7:31:01,  6.90s/it] 